<a href="https://colab.research.google.com/github/skauntey/ALMS-II-sentiment-analysis/blob/main/Balanced_ALMS_Model_2_250421.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil
import kaggle
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

## 2. Downloading kaggle competition files


In [ ]:
dataset = "sentiment-analysis-on-movie-reviews"
def kaggle_dataset_download(dataset):
    kaggle.api.authenticate()
    kaggle.api.competition_download_files(dataset, path= str(os.getcwd())+"/dataset/", force = True)
    print ('Data downloaded!')

kaggle_dataset_download(dataset)

In [ ]:
## In order to download the dataset make sure that kaggle.json file is available in ~/.kaggle/kaggle.json
# 'Kaggle.json' Downloading instructions are available under 'API Credential' heading of the github page
# https://github.com/Kaggle/kaggle-api

import zipfile
#Define file to download

dataset = "sentiment-analysis-on-movie-reviews"

def kaggle_dataset_download(dataset):
    kaggle.api.authenticate()
    kaggle.api.competition_download_files(dataset, path= str(os.getcwd())+"/dataset/", force = True)
    print ('Data downloaded!')

def unzip_kaggle_file():

    # file_path
    zipped_file = r'dataset/sentiment-analysis-on-movie-reviews.zip'
    file_path = os.path.join(str(os.getcwd()),zipped_file)
    folder_path = os.path.join(str(os.getcwd()), 'dataset')

    # making sure that previously downloaded file, if any, is deleted before
    for file in os.listdir(folder_path):
        path = os.path.join(os.getcwd(), 'dataset', file)
        if not path.split('.')[-1] == "zip":
          shutil.rmtree(path)
        else:
          continue

    # Unzipping the file
    with zipfile.ZipFile(file_path) as zip_file:
        for member in zip_file.namelist():
            if member.split('.')[-1] == "zip":
                fdir = member.split('.')[0]
                zip_file.extract(member, path= os.path.join('dataset/'+fdir))
                # extracting individual train and test files in their respective folders
                zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
                zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
                with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
                    tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))

                os.remove(zippedfile)
            else:
                continue

    os.remove(zipped_file)
    print('Kaggle file is downloaded and unzipped!')


kaggle_dataset_download(dataset)
unzip_kaggle_file()

## 3. Converting files to DataFrame

In [ ]:
#C:\Users\eq\Documents\UCL\ELEC0135 ALMS II\ALMS-II-sentiment-analysis\dataset\train
file_path = os.path.join(str(os.getcwd()), 'dataset','train')
file = os.listdir(file_path)[0]
path = os.path.join(file_path, file)
# read the data
train_data = pd.read_csv(path, sep="\t")

pdf = pd.DataFrame({ 
    'text_reviews': train_data['Phrase'],
    'class_id': train_data['Sentiment']})

pdf_data = pdf['text_reviews']
pdf_labels = pdf['class_id']

print(pdf_data.shape)
print(pdf_labels.shape)
pdf.head(10)

#### 3.1 Viewing data

In [ ]:
sen_numbers = []
for i in pdf_f['SentenceId']:
  if i not in sen_numbers:
    sen_numbers.append(i)

len(sen_numbers)

In [ ]:
Label_Class = [('negative', 0), ('somewhat negative', 1), ('neutral', 2), ('somewhat positive', 3), ('positive', 4)]
Label_Class[0][0]
pdf_f = train_data.copy()

pdf['class'] = [Label_Class[i][0] for i in pdf['class_id']]
pdf_f['sentence_len'] = [len(x) for x in train_data['Phrase']]

#for i in range(len(set(pdf_f.SentenceId))):
strings  = []
for i in sen_numbers:
  anew = pdf_f.loc[pdf_f['SentenceId'] == i].sort_values('sentence_len',ascending=False)
  seriess = anew[['Phrase', 'Sentiment', 'sentence_len']].iloc[0,0:3].values
  strings.append(seriess)

strings

In [ ]:
k= pd.DataFrame(strings, columns=['text_reviews', 'class_id', 'sentence_len'])

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
k.groupby('class_id').text_reviews.count().plot.bar(ylim=0)
plt.show()

In [ ]:
pdf_f.loc[pdf_f['SentenceId'] == 3]

In [ ]:
pdf.info()

In [ ]:
col = ['text_reviews','class_id','class']
pdf[col]

In [ ]:
from io import StringIO
id_class_df = pdf[['class_id', 'class']].drop_duplicates().sort_values('class_id')
id_class = dict(id_class_df.values)
class_id = dict(id_class_df[['class', 'class_id']].values)
class_id

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
pdf.groupby('class').text_reviews.count().plot.bar(ylim=0)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(pdf.Text_Reviews).toarray()
labels = pdf.Labels
features.shape